In [3]:
import pandas as pd
import numpy as np
import altair as alt

import geopandas as gpd
from geopandas import GeoDataFrame
import cenpy as cen
import datetime as dt

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

### imports

In [4]:
#import race data for tracts
#https://data.census.gov/cedsci/table?q=race
dfRace = pd.read_csv('CensusRaceData_cleaned.csv')

#filter to just San Francisco
dfRace = dfRace[dfRace.tract.str.contains('San Francisco')]

#create GEOID
dfRace['GEOID'] = dfRace['id'].str[10:]

In [5]:
#import census tracts
#https://data.sfgov.org/Geographic-Locations-and-Boundaries/Census-2020-Tracts-for-San-Francisco/tmph-tgz9
dfTracts = gpd.read_file('https://data.sfgov.org/api/geospatial/tmph-tgz9?method=export&format=Shapefile')

#turn GEOID into object
dfTracts['GEOID'] = dfTracts['geoid'].astype(str).str[1:]

#remove unnecessary columns
dfTracts = dfTracts[['namelsad','GEOID','geometry']]

### filter race data into neat columns

In [6]:
#include everyone with every element 
dfRace['nonh_some_white'] = dfRace.loc[:,dfRace.columns.str.contains('white')].sum(axis=1)
dfRace['nonh_some_black'] = dfRace.loc[:,dfRace.columns.str.contains('black')].sum(axis=1)
dfRace['nonh_some_native'] = dfRace.loc[:,dfRace.columns.str.contains('native')].sum(axis=1)
dfRace['nonh_some_asian'] = dfRace.loc[:,dfRace.columns.str.contains('asian')].sum(axis=1)
dfRace['nonh_some_hawaiianorpacificislander'] = dfRace.loc[:,dfRace.columns.str.contains('hawaiianorpacificislander')].sum(axis=1)
dfRace['nonh_some_other'] = dfRace.loc[:,dfRace.columns.str.contains('other')].sum(axis=1)

#hone down to mixed race categories
dfRaceAll = dfRace[['GEOID','total_pop','hispanic','nonh_some_white','nonh_some_black','nonh_some_native','nonh_some_asian','nonh_some_hawaiianorpacificislander','nonh_some_other']]

### merge

In [7]:
#merge
dfRaceTracts = dfTracts.merge(dfRaceAll, on='GEOID')

### chop out water/clean up geometries

#### load current supervisor map

In [10]:
#load current supervisor map
dfCurrentMap = gpd.read_file('s7rh6q.shp')

#get rid of unnecessary columns
dfCurrentMap = dfCurrentMap[['supervisor','geometry']]

#set correct crs
dfCurrentMap = dfCurrentMap.to_crs(4269)

In [11]:
#explode!
dfCurrentMap = dfCurrentMap.explode().reset_index().drop(['level_0','level_1'],axis=1)

#drop annoying sliver
dfCurrentMap = dfCurrentMap.drop(8)

C:\Users\willi\AppData\Local\Temp/ipykernel_14616/1036791033.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  dfCurrentMap = dfCurrentMap.explode().reset_index().drop(['level_0','level_1'],axis=1)


In [12]:
#export as geoJSON
dfCurrentMap.to_file("CurrentSupMap.geojson", driver="GeoJSON")

In [13]:
#create beautiful intersection map
dfCurrentMapCity = dfCurrentMap.dissolve()

#### cut everything down to current sup map size

In [14]:
#get rid of tracts without any population
dfRaceTracts = dfRaceTracts[dfRaceTracts['total_pop'] != 0]

In [15]:
#chop off all tracts not within the supervisor map
dfRaceTracts = dfRaceTracts.overlay(dfCurrentMapCity, how='intersection')

C:\Users\willi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:2196: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: GEOGCS["WGS84(DD)",DATUM["WGS84",SPHEROID["WGS84", ...
Right CRS: EPSG:4269

  return geopandas.overlay(


In [16]:
#get rid of multipolygons
dfRaceTracts = dfRaceTracts.explode().reset_index().drop(['level_0','level_1'],axis=1)

C:\Users\willi\AppData\Local\Temp/ipykernel_14616/171662067.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  dfRaceTracts = dfRaceTracts.explode().reset_index().drop(['level_0','level_1'],axis=1)


In [17]:
#plot map
alt.Chart(dfRaceTracts).mark_geoshape(
    fill='lightgray',
    stroke='darkgray',
    strokeWidth=0.5
).properties(
    width=700,
    height=350
).encode(
    tooltip='supervisor'
).configure_view(strokeWidth=0)

alt.Chart(...)

### bin and clean and export

In [18]:
#make mini geoJSON files
gdfHispanic = dfRaceTracts[['namelsad','total_pop','hispanic','geometry']]
gdfWhite = dfRaceTracts[['namelsad','total_pop','nonh_some_white','geometry']]
gdfBlack = dfRaceTracts[['namelsad','total_pop','nonh_some_black','geometry']]
gdfNative = dfRaceTracts[['namelsad','total_pop','nonh_some_native','geometry']]
gdfAsian = dfRaceTracts[['namelsad','total_pop','nonh_some_asian','geometry']]
gdfHawaii = dfRaceTracts[['namelsad','total_pop','nonh_some_hawaiianorpacificislander','geometry']]
gdfOther = dfRaceTracts[['namelsad','total_pop','nonh_some_other','geometry']]

In [19]:
#figure out population proportions
gdfHispanic['population_prop'] = round(gdfHispanic['hispanic'] / gdfHispanic['total_pop'] * 100, 1)
gdfWhite['population_prop'] = round(gdfWhite['nonh_some_white'] / gdfWhite['total_pop'] * 100, 1)
gdfBlack['population_prop'] = round(gdfBlack['nonh_some_black'] / gdfBlack['total_pop'] * 100, 1)
gdfNative['population_prop'] = round(gdfNative['nonh_some_native'] / gdfNative['total_pop'] * 100, 1)
gdfAsian['population_prop'] = round(gdfAsian['nonh_some_asian'] / gdfAsian['total_pop'] * 100, 1)
gdfHawaii['population_prop'] = round(gdfHawaii['nonh_some_hawaiianorpacificislander'] / gdfHawaii['total_pop'] * 100, 1)
gdfOther['population_prop'] = round(gdfOther['nonh_some_other'] / gdfOther['total_pop'] * 100, 1)

C:\Users\willi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [20]:
#only doing four racial categories for now
#bin population proportions
gdfHispanic['population_prop_bin'] = pd.cut(gdfHispanic['population_prop'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=['0-10%','10-20%','20-30%','30-40%','40-50%','50-60%','60-70%','70-80%','80-90%','90-100%'])
gdfWhite['population_prop_bin'] = pd.cut(gdfWhite['population_prop'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=['0-10%','10-20%','20-30%','30-40%','40-50%','50-60%','60-70%','70-80%','80-90%','90-100%'])
gdfBlack['population_prop_bin'] = pd.cut(gdfBlack['population_prop'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=['0-10%','10-20%','20-30%','30-40%','40-50%','50-60%','60-70%','70-80%','80-90%','90-100%'])
gdfAsian['population_prop_bin'] = pd.cut(gdfAsian['population_prop'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=['0-10%','10-20%','20-30%','30-40%','40-50%','50-60%','60-70%','70-80%','80-90%','90-100%'])

#as strings
gdfHispanic['population_prop_bin'] = gdfHispanic['population_prop_bin'].astype(str)
gdfWhite['population_prop_bin'] = gdfWhite['population_prop_bin'].astype(str)
gdfBlack['population_prop_bin'] = gdfBlack['population_prop_bin'].astype(str)
gdfAsian['population_prop_bin'] = gdfAsian['population_prop_bin'].astype(str)

In [21]:
gdfHispanic.to_file("gdfHispanic.geojson", driver="GeoJSON")
gdfWhite.to_file("gdfWhite.geojson", driver="GeoJSON")
gdfBlack.to_file("gdfBlack.geojson", driver="GeoJSON")
gdfAsian.to_file("gdfAsian.geojson", driver="GeoJSON")

### load Map 4A shapefile

In [22]:
#load final district map
#https://sf.gov/resource/2022/shapefiles-redistricting-task-force
dfMap4A = gpd.read_file('Draft_Final_Map.shp')

In [23]:
#get rid of excess columns
dfMap4A = dfMap4A[['DISTRICT','POPULATION','geometry']]

In [24]:
#chop off all tracts not within the supervisor map
dfMap4A = dfMap4A.overlay(dfCurrentMapCity, how='intersection')

In [25]:
#get rid of multipolygons
dfMap4A = dfMap4A.explode().reset_index().drop(['level_0','level_1'],axis=1)

C:\Users\willi\AppData\Local\Temp/ipykernel_14616/3744791432.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  dfMap4A = dfMap4A.explode().reset_index().drop(['level_0','level_1'],axis=1)


In [26]:
#plot map
alt.Chart(dfMap4A).mark_geoshape(
    fill='lightgray',
    stroke='darkgray',
    strokeWidth=0.5
).properties(
    width=700,
    height=350
).configure_view(strokeWidth=0)

alt.Chart(...)

In [27]:
#export as geoJSON
dfMap4A.to_file("FinalDraftMap.geojson", driver="GeoJSON")

#### create Map 4A labels

In [ ]:
#create label points dataframe
dfMap4ALabels = [{'name': '1', 'latitude': 37.77913997009093, 'longitude': -122.48323943404117},
                {'name': '2', 'latitude': 37.79379810348995, 'longitude': -122.45129308342577},
                {'name': '3', 'latitude': 37.798289633908375, 'longitude': -122.41215762996562},
                {'name': '4', 'latitude': 37.75131347352852, 'longitude': -122.49353420763208},
                {'name': '5', 'latitude': 37.781415263711864, 'longitude': -122.42556748889184},
                {'name': '6', 'latitude': 37.77909339877755, 'longitude': -122.39962883000551},
                {'name': '7', 'latitude': 37.74303427929667, 'longitude': -122.45942975504916},
                {'name': '8', 'latitude': 37.754962290530614, 'longitude': -122.43456520332234},
                {'name': '9', 'latitude': 37.74643115886766, 'longitude': -122.41240960359413},
                {'name': '10', 'latitude': 37.73469372001913, 'longitude': -122.38781089319022},
                {'name': '11', 'latitude': 37.718526283728515, 'longitude': -122.44324447517572}]

dfMap4ALabels = pd.DataFrame(dfMap4ALabels)

In [ ]:
#turn into geodataframe
gdfMap4ALabels = gpd.GeoDataFrame(dfMap4ALabels, geometry=gpd.points_from_xy(dfMap4ALabels.longitude, dfMap4ALabels.latitude))

In [ ]:
#get rid of unnecessary columns
gdfMap4ALabels = gdfMap4ALabels[['name','geometry']]

In [ ]:
#export as geoJSON
gdfMap4ALabels.to_file("map4ALabels.geojson", driver="GeoJSON")

### create median income tracts

In [ ]:
#load in data
dfIncome = pd.read_csv('median_household_income_acs.csv')

#create GEOID
dfIncome['GEOID'] = dfIncome['id'].str[10:]

#merge
dfIncomeTracts = dfTracts.merge(dfIncome, on='GEOID')

#drop unnecessary columns
dfIncomeTracts = dfIncomeTracts[['namelsad','GEOID','estimated_household_income','estimated_household_income_moe','geometry']]

In [ ]:
#chop off all tracts not within the supervisor map
dfIncomeTracts = dfIncomeTracts.overlay(dfCurrentMapCity, how='intersection')

In [ ]:
#get rid of multipolygons
dfIncomeTracts = dfIncomeTracts.explode().reset_index().drop(['level_0','level_1'],axis=1)

In [ ]:
#bin incomes
dfIncomeTracts['income_bin'] = pd.cut(dfIncomeTracts['estimated_household_income'], [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, 1000000], labels=['0-20000','20000-40000','40000-60000','60000-80000','80000-100000','100000-120000','120000-140000','140000-160000','160000-180000','180000-200000','200000+'])

#as strings
dfIncomeTracts['income_bin'] = dfIncomeTracts['income_bin'].astype(str)

In [ ]:
#plot map
alt.Chart(dfIncomeTracts).mark_geoshape(
    stroke='darkgray',
    strokeWidth=0.5
).properties(
    width=700,
    height=350
).encode(
    tooltip='namelsad',
    color='estimated_household_income:Q'
).configure_view(strokeWidth=0)

In [ ]:
#export as geoJSON
dfIncomeTracts.to_file("dfIncomeTracts.geojson", driver="GeoJSON")